## Read data from database

In [1]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [3]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [4]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [7]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where a.section_name = '.text' and c.mw_num_engines <> -1 and (c.mw_num_engines > 6 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201701')
group by mw_file_hash
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

--- 3.104015350341797 seconds ---
--- 3.212686777114868 seconds ---
--- 3.265317916870117 seconds ---
--- 3.009309768676758 seconds ---
--- 2.8050670623779297 seconds ---
--- 2.946202516555786 seconds ---
--- 3.236978769302368 seconds ---
--- 2.8308823108673096 seconds ---
--- 2.8081748485565186 seconds ---
--- 3.0357909202575684 seconds ---
--- 2.600412607192993 seconds ---
--- 2.379838228225708 seconds ---
--- 2.6680760383605957 seconds ---
--- 1.8274145126342773 seconds ---
--- 1.811929702758789 seconds ---
--- 1.8530206680297852 seconds ---
127160


In [9]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where a.section_name = '.text' and c.mw_num_engines <> -1 and (c.mw_num_engines > 6 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201702')
group by mw_file_hash
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

--- 4.697957992553711 seconds ---
--- 7.296715259552002 seconds ---
--- 4.503918170928955 seconds ---
--- 4.409035921096802 seconds ---
--- 4.349301099777222 seconds ---
--- 4.44843316078186 seconds ---
--- 4.423307657241821 seconds ---
--- 4.492370128631592 seconds ---
--- 4.422391176223755 seconds ---
--- 6.929706811904907 seconds ---
--- 4.831637144088745 seconds ---
--- 4.639912843704224 seconds ---
--- 4.666112899780273 seconds ---
--- 4.807678461074829 seconds ---
--- 5.224670171737671 seconds ---
--- 5.1271138191223145 seconds ---
84582


## Check and split data

In [33]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 300000

train_data = pd.DataFrame(res1)
train_data = train_data.loc[train_data.virtual_size <= max_length]
train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines > 6 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
test_data = test_data.loc[test_data.virtual_size <= max_length]
test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines > 6 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ip

## EMBER

In [34]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

In [35]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

In [36]:
def get_model(x_train, y_train, x_val, y_val):
    params = {'application': 'binary'}
    lgbm_dataset = lgb.Dataset(x_train, y_train.ravel())
    valid_sets = lgb.Dataset(x_val, y_val.ravel())

    model = lgb.train(params, lgbm_dataset, 100000, valid_sets=valid_sets, early_stopping_rounds=10)
    y_pred = model.predict(x_val)
    
    loss = log_loss(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
#     model.save_model(file_path + "-%04d-%.5f-%.5f.h5" % (model.best_iteration, loss, acc),
#                      num_iteration=model.best_iteration)
    print("val loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)
        
    return model

In [37]:
x_etrain = get_ember_feature(x_train)
x_eval = get_ember_feature(x_val)
x_etest = get_ember_feature(x_test)

In [40]:
import matplotlib.pyplot as plt
model = get_model(x_etrain, y_train, x_eval, y_val)

[1]	valid_0's binary_logloss: 0.611326
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.545228
[3]	valid_0's binary_logloss: 0.489771
[4]	valid_0's binary_logloss: 0.442264
[5]	valid_0's binary_logloss: 0.401525
[6]	valid_0's binary_logloss: 0.365412
[7]	valid_0's binary_logloss: 0.333953
[8]	valid_0's binary_logloss: 0.307177
[9]	valid_0's binary_logloss: 0.282782
[10]	valid_0's binary_logloss: 0.261964
[11]	valid_0's binary_logloss: 0.242456
[12]	valid_0's binary_logloss: 0.225985
[13]	valid_0's binary_logloss: 0.210507
[14]	valid_0's binary_logloss: 0.197181
[15]	valid_0's binary_logloss: 0.184133
[16]	valid_0's binary_logloss: 0.173379
[17]	valid_0's binary_logloss: 0.162612
[18]	valid_0's binary_logloss: 0.153024
[19]	valid_0's binary_logloss: 0.144421
[20]	valid_0's binary_logloss: 0.136783
[21]	valid_0's binary_logloss: 0.129512
[22]	valid_0's binary_logloss: 0.122847
[23]	valid_0's binary_logloss: 0.11697
[24]	valid_0's binary_loglos

[199]	valid_0's binary_logloss: 0.0243037
[200]	valid_0's binary_logloss: 0.0242221
[201]	valid_0's binary_logloss: 0.0242247
[202]	valid_0's binary_logloss: 0.0241727
[203]	valid_0's binary_logloss: 0.0240803
[204]	valid_0's binary_logloss: 0.024034
[205]	valid_0's binary_logloss: 0.023964
[206]	valid_0's binary_logloss: 0.0238902
[207]	valid_0's binary_logloss: 0.023861
[208]	valid_0's binary_logloss: 0.023862
[209]	valid_0's binary_logloss: 0.0238614
[210]	valid_0's binary_logloss: 0.0237772
[211]	valid_0's binary_logloss: 0.0237241
[212]	valid_0's binary_logloss: 0.0237328
[213]	valid_0's binary_logloss: 0.0237472
[214]	valid_0's binary_logloss: 0.0237391
[215]	valid_0's binary_logloss: 0.0237735
[216]	valid_0's binary_logloss: 0.0237247
[217]	valid_0's binary_logloss: 0.0237179
[218]	valid_0's binary_logloss: 0.0236478
[219]	valid_0's binary_logloss: 0.0236075
[220]	valid_0's binary_logloss: 0.0235903
[221]	valid_0's binary_logloss: 0.0236002
[222]	valid_0's binary_logloss: 0.0235

In [41]:
def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

In [42]:
y_p = model.predict(x_etest)
y_pred_e = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred_e[i, 0] = y_p[i]

estimate_model(y_pred_e, y_test)

loss : 0.24118
auc score : 0.98498
accuracy score : 0.96377
thre: 0.9999957052
fp:  0.0009623157
recall:  0.0825126263


(0.9849844963890804, 0.24117966536573404, 0.08251262626262626)

## Malcon

In [43]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, labels, batch_size=32, dim=8192, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)
        y = np.zeros(self.batch_size, dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data
            y[i] = self.labels[ID]

        return X, y

In [30]:
import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Multiply, GlobalMaxPooling1D, Dropout
from keras.models import load_model
from sklearn.model_selection import train_test_split

class TMalConv(object):
    """
    train of mal conv
    """

    def __init__(self):
        self.max_len = max_length
        self.history = None
        self.model = None
        self.p_md5 = None
        self.time = time.time()
        self.summary = {
            'time':time.time(),
            'batch_size': 32,
            'epochs': 64,
            'g_c_filter': 128,
            'g_c_kernel_size': 500,
            'g_c_stride': 500,
        }

    def run(self):
        """
        :return:
        """
        self.train()
        
    def get_p(self, key):
        """
        get the parameter from the summary
        :param key:
        :return:
        """
        return self.summary[key]

    def gate_cnn(self, gate_cnn_input):
        """
        construct a gated cnn by the specific kernel size
        :param gate_cnn_input:
        :param kernel_size:
        :return:
        """
        conv1_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"))(
            gate_cnn_input)
        conv2_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"),
                           activation="sigmoid")(gate_cnn_input)
        merged = Multiply()([conv1_out, conv2_out])
        gate_cnn_output = GlobalMaxPooling1D()(merged)
        return gate_cnn_output

    def get_model(self):
        """
        get a model
        :param max_len:
        :param kernel_sizes:
        :return:
        """
        net_input = Input(shape=(self.max_len,))

        embedding_out = Embedding(256, 8, input_length=self.max_len)(net_input)
        merged = self.gate_cnn(embedding_out)

        dense_out = Dense(128)(merged)
        
        net_output = Dense(1, activation='sigmoid')(dense_out)

        model = keras.models.Model(inputs=net_input, outputs=net_output)

        return model

    def train(self):
        batch_size = self.get_p("batch_size")
        epochs = self.get_p("epochs")

        self.model = self.get_model()

        print('Length of the train: ', len(x_train))
        print('Length of the validation: ', len(x_val))
        
#         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/BaseTrain/models/"+ str(self.time) +"-{epoch:04d}-{val_loss:.5f}-{val_acc:.5f}.h5"
        early_stopping = EarlyStopping("val_loss", patience=3, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point, early_stopping]

        # Generators
        training_generator = DataGenerator(range(len(x_train)), x_train, y_train, batch_size, self.max_len)
        validation_generator = DataGenerator(range(len(x_val)), x_val, y_val, batch_size, self.max_len)

        self.model.compile(loss='binary_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])

        self.model.fit_generator(generator=training_generator,
                                 validation_data=validation_generator,
                                 use_multiprocessing=True,
                                 epochs=epochs,
                                 workers=6,
                                 callbacks=callbacks_list)

In [44]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [45]:
t_instance = TMalConv()
t_instance.run()

Length of the train:  93077
Length of the validation:  10342
Epoch 1/64
2908/2908 [==============================] - 952s 327ms/step - loss: 0.1620 - acc: 0.9376 - val_loss: 0.1357 - val_acc: 0.9500

Epoch 00001: val_loss improved from inf to 0.13568, saving model to /home/zhaoqi/BaseTrain/models/1531997458.8325162-0001-0.13568-0.94998.h5
Epoch 2/64
2908/2908 [==============================] - 1190s 409ms/step - loss: 0.0944 - acc: 0.9706 - val_loss: 0.1249 - val_acc: 0.9541

Epoch 00002: val_loss improved from 0.13568 to 0.12492, saving model to /home/zhaoqi/BaseTrain/models/1531997458.8325162-0002-0.12492-0.95414.h5
Epoch 3/64
2908/2908 [==============================] - 1500s 516ms/step - loss: 0.0651 - acc: 0.9851 - val_loss: 0.1314 - val_acc: 0.9550

Epoch 00003: val_loss did not improve from 0.12492
Epoch 4/64
2908/2908 [==============================] - 1499s 515ms/step - loss: 0.0571 - acc: 0.9884 - val_loss: 0.1428 - val_acc: 0.9489

Epoch 00004: val_loss did not improve from 

In [46]:
model_dir = '/home/zhaoqi/BaseTrain/models/'
f_name = '1531997458.8325162-0002-0.12492-0.95414.h5'
c_model = load_model(model_dir + f_name)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
y_pred = c_model.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

2118/2118 [==============================] - 867s 409ms/step


In [47]:
estimate_model(y_pred, y_test[0:len(y_pred)])

loss : 0.26196
auc score : 0.97068
accuracy score : 0.93539
thre: 0.9997478127
fp:  0.0009626121
recall:  0.0410509031


(0.9706777008840716, 0.26196404833713366, 0.041050903119868636)

## Merge features

### Merge Malconv and Ember

In [48]:
from keras.models import Model

model_f = Model(c_model.input, c_model.layers[-2].output)

train_generator = DataGenerator(range(len(x_train)), x_train, y_train, 32, max_length, False)
malcon_train_x = model_f.predict_generator(generator=train_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

val_generator = DataGenerator(range(len(x_val)), x_val, y_val, 32, max_length, False)
malcon_val_x = model_f.predict_generator(generator=val_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
malcon_test_x = model_f.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

2118/2118 [==============================] - 934s 441ms/step


In [49]:
model_m = get_model(malcon_train_x, y_train[0:len(malcon_train_x)], malcon_val_x, y_val[0:len(malcon_val_x)] )

[1]	valid_0's binary_logloss: 0.608134
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.53956
[3]	valid_0's binary_logloss: 0.482521
[4]	valid_0's binary_logloss: 0.434895
[5]	valid_0's binary_logloss: 0.394731
[6]	valid_0's binary_logloss: 0.360475
[7]	valid_0's binary_logloss: 0.330977
[8]	valid_0's binary_logloss: 0.305727
[9]	valid_0's binary_logloss: 0.283994
[10]	valid_0's binary_logloss: 0.265143
[11]	valid_0's binary_logloss: 0.248652
[12]	valid_0's binary_logloss: 0.234477
[13]	valid_0's binary_logloss: 0.222189
[14]	valid_0's binary_logloss: 0.211543
[15]	valid_0's binary_logloss: 0.202204
[16]	valid_0's binary_logloss: 0.194232
[17]	valid_0's binary_logloss: 0.187318
[18]	valid_0's binary_logloss: 0.181133
[19]	valid_0's binary_logloss: 0.175902
[20]	valid_0's binary_logloss: 0.171292
[21]	valid_0's binary_logloss: 0.167301
[22]	valid_0's binary_logloss: 0.163888
[23]	valid_0's binary_logloss: 0.161171
[24]	valid_0's binary_loglos

In [50]:
y_p = model_m.predict(malcon_test_x)
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, y_test[0:len(malcon_test_x)])

loss : 0.24731
auc score : 0.97065
accuracy score : 0.92909
thre: 0.9863750093
fp:  0.0000770090
recall:  0.0003157762


(0.970649811030129, 0.24731460953844853, 0.0003157761778451434)

In [51]:
def merge_feature(m_data, e_data):
    num = len(m_data)
    m_x = np.zeros((num, 128+2351), dtype=float)
    
    for index in range(num):
        m_x[index, 0:128] = m_data[index]
        m_x[index, 128:128+2351] = e_data[index]  
    return m_x

In [52]:
merge_train_x = merge_feature(malcon_train_x, x_etrain)
merge_val_x = merge_feature(malcon_val_x, x_eval)
merge_test_x = merge_feature(malcon_test_x, x_etest)

model_m = get_model(merge_train_x, y_train[0:len(merge_train_x)], merge_val_x, y_val[0:len(merge_val_x)] )

[1]	valid_0's binary_logloss: 0.605535
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.534004
[3]	valid_0's binary_logloss: 0.474748
[4]	valid_0's binary_logloss: 0.424647
[5]	valid_0's binary_logloss: 0.382659
[6]	valid_0's binary_logloss: 0.346437
[7]	valid_0's binary_logloss: 0.315354
[8]	valid_0's binary_logloss: 0.287779
[9]	valid_0's binary_logloss: 0.263552
[10]	valid_0's binary_logloss: 0.242721
[11]	valid_0's binary_logloss: 0.224152
[12]	valid_0's binary_logloss: 0.2076
[13]	valid_0's binary_logloss: 0.193193
[14]	valid_0's binary_logloss: 0.180624
[15]	valid_0's binary_logloss: 0.169606
[16]	valid_0's binary_logloss: 0.159523
[17]	valid_0's binary_logloss: 0.150378
[18]	valid_0's binary_logloss: 0.142689
[19]	valid_0's binary_logloss: 0.135555
[20]	valid_0's binary_logloss: 0.129519
[21]	valid_0's binary_logloss: 0.124052
[22]	valid_0's binary_logloss: 0.119223
[23]	valid_0's binary_logloss: 0.115013
[24]	valid_0's binary_logloss

In [53]:
y_p = model_m.predict(merge_test_x)
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, y_test[0:len(merge_test_x)])

loss : 0.25190
auc score : 0.98125
accuracy score : 0.94895
thre: 0.9995773224
fp:  0.0000000000
recall:  0.0000000000


(0.9812466612240963, 0.251895966677737, 0.0)